In [1]:
from phi35.utils import *
tokenizer = load_tokenizer()
hidden_states = get_hidden_states(PROMPT, tokenizer)
embeddings_table = get_token_embeddings_table()
umap_model = get_umap_model(embeddings_table)
umap_2d = get_2d_representation(embeddings_table, umap_model)
token_ids = get_prompt_token_ids(PROMPT, tokenizer)

Loading tokenizer...
Tokenizer loaded successfully.
Getting hidden states. Recalculate: False
Loading cached output


c:\Users\johan_2po46\code\HowTransformersTransform\.venv\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b

Retrieving token embeddings table...
Loading existing token embeddings table
Token embeddings table shape: (32064, 3072)
Getting UMAP model. Recalculate: False
Loading existing UMAP model
Loading existing 2D representation
2D representation shape: (32064, 2)
Tokenizing prompt: 'An android named Apple was los...'
Prompt tokenized. Number of tokens: 182


In [36]:
token_ids = tokenizer.encode(PROMPT, add_special_tokens=False)
tokens = tokenizer.convert_ids_to_tokens(token_ids)
tokens = [token.replace("▁", " ") for token in tokens if token != "<0x0A>"]


In [38]:
[token.replace("▁", " ") for token in tokens]


[' An',
 ' android',
 ' named',
 ' Apple',
 ' was',
 ' lost',
 ' in',
 ' the',
 ' woods',
 '.',
 ' L',
 'uck',
 'ily',
 ',',
 ' he',
 ' had',
 ' a',
 ' phone',
 ' with',
 ' Google',
 ' Maps',
 '.',
 ' As',
 ' he',
 ' pulled',
 ' out',
 ' the',
 ' device',
 ',',
 ' its',
 ' screen',
 ' f',
 'lick',
 'ered',
 ' to',
 ' life',
 ',',
 ' casting',
 ' a',
 ' faint',
 ' blue',
 ' g',
 'low',
 ' on',
 ' his',
 ' met',
 'all',
 'ic',
 ' face',
 '.',
 ' Apple',
 "'",
 's',
 ' optical',
 ' sens',
 'ors',
 ' sc',
 'anned',
 ' the',
 ' map',
 ',',
 ' processing',
 ' the',
 ' terrain',
 ' data',
 '.',
 '"',
 'Calcul',
 'ating',
 ' optimal',
 ' route',
 ',"',
 ' he',
 ' mut',
 'tered',
 ',',
 ' his',
 ' voice',
 ' a',
 ' mel',
 'od',
 'ic',
 ' hum',
 ' of',
 ' circ',
 'uits',
 ' and',
 ' synth',
 'es',
 'ized',
 ' speech',
 '.',
 'S',
 'ud',
 'den',
 'ly',
 ',',
 ' a',
 ' drop',
 ' of',
 ' water',
 ' spl',
 'ashed',
 ' onto',
 ' the',
 ' screen',
 '.',
 ' Then',
 ' another',
 '.',
 ' Apple',
 ' looke

: 

{'PRO': 8618,
 '▁grown': 21633,
 '▁Nevertheless': 25678,
 'richt': 10220,
 '▁Indians': 17072,
 '▁hö': 12977,
 '▁substantial': 23228,
 'otlin': 13961,
 'ock': 1698,
 ",'": 5501,
 'nc': 17608,
 '▁commits': 25741,
 'iams': 10909,
 '▁Vall': 19918,
 '▁apare': 13059,
 'pg': 4061,
 '▁looping': 26113,
 '▁fri': 3484,
 'UMN': 29127,
 '▁Bes': 6952,
 '\\}$,': 28538,
 'nach': 29683,
 '▁neighbourhood': 29643,
 '▁Fourier': 27506,
 '▁méd': 10283,
 '▁suddenly': 11584,
 '▁though': 2466,
 'diff': 12765,
 '|$': 25183,
 '▁descend': 17086,
 'scher': 19996,
 '▁can': 508,
 'ნ': 30632,
 '▁like': 763,
 '▁Vo': 4785,
 '▁poll': 21180,
 'vest': 10147,
 '▁Ана': 22695,
 'kirche': 22132,
 'Miss': 18552,
 '▁voet': 20613,
 'avait': 23071,
 '▁NGC': 13928,
 '時': 30974,
 '▁read': 1303,
 '▁ign': 5330,
 'хі': 22177,
 '▁anno': 12327,
 '₈': 30278,
 'լ': 31235,
 '▁;': 2056,
 '▁creation': 11265,
 'iej': 8586,
 '▁Cra': 14279,
 'Arr': 16401,
 'стем': 18773,
 'elled': 14356,
 '▁inequality': 14585,
 'allery': 23365,
 '▁Milano': 1635

In [6]:
for token in tokenizer.convert_ids_to_tokens(token_ids):
    print(token)


▁An
▁android
▁named
▁Apple
▁was
▁lost
▁in
▁the
▁woods
.
▁L
uck
ily
,
▁he
▁had
▁a
▁phone
▁with
▁Google
▁Maps
.
▁As
▁he
▁pulled
▁out
▁the
▁device
,
▁its
▁screen
▁f
lick
ered
▁to
▁life
,
▁casting
▁a
▁faint
▁blue
▁g
low
▁on
▁his
▁met
all
ic
▁face
.
▁Apple
'
s
▁optical
▁sens
ors
▁sc
anned
▁the
▁map
,
▁processing
▁the
▁terrain
▁data
.
<0x0A>
"
Calcul
ating
▁optimal
▁route
,"
▁he
▁mut
tered
,
▁his
▁voice
▁a
▁mel
od
ic
▁hum
▁of
▁circ
uits
▁and
▁synth
es
ized
▁speech
.
<0x0A>
S
ud
den
ly
,
▁a
▁drop
▁of
▁water
▁spl
ashed
▁onto
▁the
▁screen
.
▁Then
▁another
.
▁Apple
▁looked
▁up
,
▁ra
ind
ro
ps
▁now
▁pat
ter
ing
▁against
▁his
▁water
proof
▁cas
ing
.
▁The
▁weather
▁was
▁inter
fer
ing
▁with
▁his
▁G
PS
▁signal
.
<0x0A>
As
▁the
▁down
p
our
▁intens
ified
,
▁Apple
▁realized
▁he
▁faced
▁a
▁new
▁challenge
.
▁His
▁circuit
ry
▁was
▁protected
,
▁but
▁the
▁phone
▁wasn
'
t
▁water
proof
.
▁He
▁needed
▁shelter
,
▁and
▁fast
.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from io import BytesIO
import base64
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display

def get_color(similarity):
    cmap = colors.LinearSegmentedColormap.from_list("", ["red", "yellow", "green"])
    return colors.rgb2hex(cmap(similarity))

def get_min_max_similarity_except_self(emb, embeddings):
    prompt_similarities = cosine_similarity(emb, embeddings)
    one_mask = np.isclose(prompt_similarities, 1, atol=1e-3)
    non_one_max = np.max(prompt_similarities[~one_mask])
    prompt_max_sim = non_one_max
    prompt_min_sim = np.min(prompt_similarities)
    return prompt_min_sim, prompt_max_sim

def create_colorbar(min_sim, max_sim):
    fig, ax = plt.subplots(figsize=(6, 1))
    cmap = colors.LinearSegmentedColormap.from_list("", ["red", "yellow", "green"])
    norm = colors.Normalize(vmin=min_sim, vmax=max_sim)
    plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), cax=ax, orientation='horizontal', label='Similarity')
    plt.tight_layout()
    
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    
    img_str = base64.b64encode(buf.getvalue()).decode()
    return f'<img src="data:image/png;base64,{img_str}">'


def on_token_click(b):
    token_id = b.token_id
    emb = token_embeddings[token_id]
    similar_tokens = get_top_similar_tokens(emb, token_embeddings, n=5, decode=True, tokenizer=tokenizer)
    result.value = f"Top 5 similar tokens for '{b.description}': {', '.join(similar_tokens)}"
    
    # Reshape emb to a 2D array
    emb_reshaped = emb.reshape(1, -1)
    similarities = cosine_similarity(emb_reshaped, prompt_embeddings).flatten()
    prompt_min, prompt_max = get_min_max_similarity_except_self(emb_reshaped, prompt_embeddings)
    
    for btn, sim in zip(token_buttons, similarities):
        if btn.token_id == token_id:
            btn.style.button_color = 'lightblue'  # Highlight the clicked token
        else:
            normalized_sim = (sim - prompt_min) / (prompt_max - prompt_min)
            normalized_sim = np.clip(normalized_sim, 0, 1)
            btn.style.button_color = get_color(normalized_sim)
    
    # Update color bar
    colorbar.value = create_colorbar(prompt_min, prompt_max)


# Calculate the maximum token length
max_token_length = max(len(tokenizer.decode([token.item()])) for token in token_ids.flatten())

# Create buttons for each token with decoded text
token_buttons = []
for token in token_ids.flatten():
    decoded_token = tokenizer.decode([token.item()])
    token_length = len(decoded_token)
    
    # Calculate width based on token length, with min and max limits
    min_width = 50  # Minimum width in pixels
    max_width = 200  # Maximum width in pixels
    width = min(max(min_width, int(token_length / max_token_length * max_width)), max_width)
    
    btn = widgets.Button(
        description=decoded_token,
        layout=widgets.Layout(width=f'{width}px', height='30px')
    )
    btn.token_id = token.item()
    btn.on_click(on_token_click)
    token_buttons.append(btn)

# Create a FlexBox layout for the buttons
button_layout = widgets.Layout(flex_flow='row wrap', align_items='flex-start')
buttons_box = widgets.Box(children=token_buttons, layout=button_layout)

# Create a toggle for normalization
normalize_toggle = widgets.ToggleButton(
    value=True,
    description='Normalize',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Toggle normalization',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

# Function to handle toggle change
def on_toggle_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # Simulate a click on the last clicked token to update the view
        last_clicked = next((btn for btn in token_buttons if btn.style.button_color == 'lightblue'), None)
        if last_clicked:
            on_token_click(last_clicked)

normalize_toggle.observe(on_toggle_change, names='value')

# Create an output widget to display results
result = widgets.Label(value="Click on a token to see similar tokens")

# Create initial color bar (will be updated on click)
colorbar = widgets.HTML(create_colorbar(0, 1))

# Display the widgets and color bar
display(widgets.VBox([normalize_toggle, buttons_box, result, colorbar]))

NameError: name 'tokens' is not defined